# 使用高德地图进行行政区域查询

In [171]:
import requests
import numpy as np
import pandas as pd
from pandas import DataFrame
import json

## 使用说明
- 第一步，申请”Web服务API”密钥（Key）；
- 第二步，拼接HTTP请求URL，第一步申请的Key需作为必填参数一同发送；
- 第三步，接收HTTP请求返回的数据（JSON或XML格式），解析数据

## 行政区域查询
行政区域查询API服务地址：
http://restapi.amap.com/v3/config/district?parameters <br>
parameters代表的参数包括必填参数和可选参数。所有参数均使用和号字符(&)进行分隔<br>
具体参数细节请参考网址http://lbs.amap.com/api/webservice/guide/api/district/

In [172]:
#parameters设置
server_url = 'http://restapi.amap.com/v3/config/district?'
key = 'key=a5fd524a333711f11bc25418a5c7a558'
subdistrict = 'subdistrict=1'
extensions = 'extensions=all'

In [200]:
def get_region(location):
    url = server_url +  'keywords=' + str(location) + '&' + subdistrict + '&' + extensions +  '&' +  key
    
    r = requests.get(url)
    temp_data = r.json()
    # 获取各行政区的地区代码，因为名字有可能会与其他城市的区重名
    adcode = []
    name = []
    for region in temp_data["districts"][1]["districts"]:
        adcode.append(region['adcode'])
        name.append(region['name'])
    
    return {'adcode': adcode, 'name': name}

In [207]:
def get_mapdata(location):
    url = server_url +  'keywords=' + str(location) + '&' + subdistrict + '&' + extensions +  '&' +  key
        
    r = requests.get(url)
    temp_data = r.json()
    
    # 获取各行政区边界经纬度数据
    #polyline数据是一整个纯文本数据，不同的地理块按照|分，块里面的地理信息按照；分，横纵坐标按照，分。
    longitude = []
    latitude = []
    join_seq = []

    region_data = temp_data["districts"][0]['polyline'].split('|')
    join_n = 1
    for per_data in region_data:
        
        point = per_data.split(';')
        for position in point:
            temp = position.split(',')
            longitude.append(temp[0])
            latitude.append(temp[1])
            join_seq.append(join_n)
            join_n = join_n + 1
    
    map_data = {'Longitude': longitude, 'Latitude': latitude, 'Seq': join_seq}
    return map_data

In [208]:
local_info = get_region('上海')
name = local_info['name']
n = 0
block_n = 1
map_data = {}
longitude = []
latitude = []
join_seq = []
region = []
block = []
#frame = DataFrame({'Longitude': [0.0], 'Latitude': [0.0], 'seq': [0], 'name': ['0'], 'block': ['0']})
for adcode in local_info['adcode']:
    temp = get_mapdata(adcode)
    length  = len(temp['Longitude'])
    
    longitude  = longitude + temp['Longitude']
    latitude = latitude + temp['Latitude']
    join_seq = join_seq + temp['Seq']
    region  = region + length*[name[n]]
    block = block + length*[str(block_n)]
    
    n = n + 1
    block_n = block_n + 1
    
frame = DataFrame({'name': region, 'Longitude': longitude, 'Latitude': latitude, 'Seq': join_seq, 'block': block})

In [209]:
frame[frame.block == '10']

,Latitude,Longitude,Seq,block,name
10044,31.240853,121.882620,1,10,浦东新区
10045,31.242595,121.889910,2,10,浦东新区
10046,31.242113,121.897368,3,10,浦东新区
10047,31.239413,121.905931,4,10,浦东新区
10048,31.236555,121.915452,5,10,浦东新区
10049,31.233862,121.923553,6,10,浦东新区
10050,31.229735,121.926732,7,10,浦东新区
10051,31.224019,121.927522,8,10,浦东新区
10052,31.205436,121.925453,9,10,浦东新区
10053,31.196858,121.922438,10,10,浦东新区


In [210]:
frame.to_csv('shanghai_region_map_data.csv', encoding='utf-8')